# MERRA-2 Soil Moisture and Precipitation Processing (CDO)

This notebook downloads and processes MERRA-2 hourly soil moisture and precipitation data for the US Lower 48.

## MERRA-2 Collection: M2T1NXLND - Land Surface Diagnostics (Hourly)

## Variables to Extract:

### Soil Moisture:
- **SFMC**: Surface soil moisture (0-5cm) (kg/m²)
- **RZMC**: Root zone soil moisture (0-100cm) (kg/m²)
- **PRMC**: Profile soil moisture (full column) (kg/m²)
- **GWETROOT**: Root zone soil wetness fraction (0-1)
- **GWETTOP**: Surface soil wetness fraction (0-1)

### Precipitation (converted to mm/hr):
- **PRECIP_TOTAL**: Total precipitation over land (from PRECTOTLAND)
- **PRECIP_SNOW**: Snowfall over land (from PRECSNOLAND)

## Output Format:
- Files: `merra2_soil_precip_us_YYYYMMDD.nc`
- Variables: SFMC, RZMC, PRMC, GWETROOT, GWETTOP, PRECIP_TOTAL, PRECIP_SNOW
- Region: US Lower 48
- Temporal: Hourly (24 timesteps per day)

## 1. Check CDO Installation

In [1]:
import subprocess
import sys

# Check if CDO is installed
try:
    result = subprocess.run(['cdo', '--version'], capture_output=True, text=True)
    print("✓ CDO is installed:")
    print(result.stdout.split('\n')[0])
except FileNotFoundError:
    print("✗ CDO is not installed!")
    print("\nInstallation instructions:")
    print("  macOS:   brew install cdo")
    print("  Ubuntu:  sudo apt-get install cdo")
    print("  conda:   conda install -c conda-forge cdo")
    sys.exit(1)

# Check for NCO (optional but helpful)
try:
    result = subprocess.run(['ncks', '--version'], capture_output=True, text=True)
    print("\n✓ NCO is installed (optional):")
    print(result.stdout.split('\n')[0])
except FileNotFoundError:
    print("\n⚠ NCO not installed (optional, but recommended for better compression)")
    print("  Install: brew install nco  (or conda install -c conda-forge nco)")

✓ CDO is installed:
Climate Data Operators version 2.5.1 (https://mpimet.mpg.de/cdo)

✓ NCO is installed (optional):



## 2. Setup and Imports

In [2]:
import earthaccess
import subprocess
from pathlib import Path
import pandas as pd
from datetime import datetime, timedelta
from tqdm.notebook import tqdm
import os

print("Libraries imported successfully")

Libraries imported successfully


## 3. Authenticate with NASA EarthData

In [3]:
# Authenticate
auth = earthaccess.login()
print("✓ Authentication successful!")

✓ Authentication successful!


## 4. Configuration

In [4]:
# US Lower 48 Bounding Box
bbox = (-125, 24, -66, 49)  # (min_lon, min_lat, max_lon, max_lat)

# CDO bbox format: lonmin,lonmax,latmin,latmax
cdo_bbox = f"{bbox[0]},{bbox[2]},{bbox[1]},{bbox[3]}"

# Date range
start_date = "1984-01-01"
end_date = "2025-12-31"

# MERRA-2 collection for Land Surface Diagnostics
collection_id = "M2T1NXLND"  # Hourly land surface diagnostics

# Output directory
output_dir = Path("daily_data_soil_precip")
output_dir.mkdir(parents=True, exist_ok=True)

# Temporary directory for intermediate files
temp_dir = Path("temp_processing_soil")
temp_dir.mkdir(parents=True, exist_ok=True)

print(f"Configuration:")
print(f"  Date range: {start_date} to {end_date}")
print(f"  Bounding box: {bbox}")
print(f"  CDO bbox: {cdo_bbox}")
print(f"  Collection: {collection_id}")
print(f"  Output directory: {output_dir}")
print(f"  Temp directory: {temp_dir}")

Configuration:
  Date range: 1984-01-01 to 2025-12-31
  Bounding box: (-125, 24, -66, 49)
  CDO bbox: -125,-66,24,49
  Collection: M2T1NXLND
  Output directory: daily_data_soil_precip
  Temp directory: temp_processing_soil


## 5. CDO Processing Functions

In [5]:
def check_file_exists(date, output_dir):
    """Check if output file already exists."""
    if isinstance(date, str):
        date = pd.to_datetime(date)
    filename = f"merra2_soil_precip_us_{date.strftime('%Y%m%d')}.nc"
    return (output_dir / filename).exists()


def download_merra2_file(date, collection_id, auth):
    """
    Download MERRA-2 file for a single day using earthaccess.
    Returns path to downloaded file or None if not found.
    """
    date_str = date.strftime('%Y-%m-%d')
    
    # Search for granule - use same date for start and end to get only that day
    results = earthaccess.search_data(
        short_name=collection_id,
        temporal=(date_str, date_str),
    )
    
    if len(results) == 0:
        return None
    
    # Download ONLY the first granule
    downloaded_files = earthaccess.download(results[0], temp_dir)
    
    if len(downloaded_files) > 0:
        return downloaded_files[0]
    return None


def process_soil_precip_cdo(input_file, output_file, cdo_bbox):
    """
    Process MERRA-2 land surface file using CDO:
    1. Subset spatial region
    2. Select all soil moisture and precipitation variables
    3. Convert precipitation from kg/m²/s to mm/hr
    4. Save as compressed float32 NetCDF
    
    Soil Moisture Layers (kg/m²):
    - SFMC: Surface layer (0-5 cm)
    - RZMC: Root zone (0-100 cm)  
    - PRMC: Profile (full column)
    - GWETROOT: Root zone wetness fraction (0-1)
    - GWETTOP: Surface wetness fraction (0-1)
    
    Precipitation variables (kg/m²/s → mm/hr):
    - PRECTOTLAND: Total precipitation over land
    - PRECSNOLAND: Snowfall over land
    
    Returns True if successful, False otherwise.
    """
    try:
        temp_base = temp_dir / f"temp_{os.getpid()}"
        
        # Step 1: Subset region and select soil moisture + precipitation variables
        subset_file = f"{temp_base}_subset.nc"
        cmd = [
            'cdo', '-f', 'nc4', '-z', 'zip_4',
            f'-sellonlatbox,{cdo_bbox}',
            '-selname,SFMC,RZMC,PRMC,GWETROOT,GWETTOP,PRECTOTLAND,PRECSNOLAND',
            input_file,
            subset_file
        ]
        subprocess.run(cmd, check=True, capture_output=True)
        
        # Step 2: Convert precipitation to mm/hr and ADD to existing variables
        # Using -expr will CREATE new variables alongside the soil moisture ones
        conversion_expr = (
            "PRECIP_TOTAL=PRECTOTLAND*3600;"
            "PRECIP_SNOW=PRECSNOLAND*3600;"
            "SFMC=SFMC;"
            "RZMC=RZMC;"
            "PRMC=PRMC;"
            "GWETROOT=GWETROOT;"
            "GWETTOP=GWETTOP;"
        )
        
        calc_file = f"{temp_base}_calc.nc"
        cmd = [
            'cdo', '-f', 'nc4', '-z', 'zip_4',
            f'-expr,{conversion_expr}',
            subset_file,
            calc_file
        ]
        subprocess.run(cmd, check=True, capture_output=True)
        
        # Step 3: Select final variables (all soil moisture + converted precipitation)
        final_file = f"{temp_base}_final.nc"
        cmd = [
            'cdo', '-f', 'nc4', '-z', 'zip_4',
            '-selname,SFMC,RZMC,PRMC,GWETROOT,GWETTOP,PRECIP_TOTAL,PRECIP_SNOW',
            calc_file,
            final_file
        ]
        subprocess.run(cmd, check=True, capture_output=True)
        
        # Step 4: Convert to float32 if NCO is available
        try:
            cmd = [
                'ncks', '-O', '-4', '--deflate', '4',
                '--ppc', 'default=5',  # 5 significant digits (float32 precision)
                final_file,
                output_file
            ]
            subprocess.run(cmd, check=True, capture_output=True)
        except (FileNotFoundError, subprocess.CalledProcessError):
            # If NCO not available, just copy the file
            import shutil
            shutil.copy(final_file, output_file)
        
        # Step 5: Add variable metadata using ncatted (if available)
        try:
            # Add units for converted variables
            subprocess.run([
                'ncatted', '-O',
                '-a', 'units,PRECIP_TOTAL,o,c,mm/hr',
                '-a', 'long_name,PRECIP_TOTAL,o,c,Total precipitation over land',
                '-a', 'description,PRECIP_TOTAL,o,c,Converted from PRECTOTLAND (kg/m2/s) to mm/hr',
                '-a', 'units,PRECIP_SNOW,o,c,mm/hr',
                '-a', 'long_name,PRECIP_SNOW,o,c,Snowfall over land',
                '-a', 'description,PRECIP_SNOW,o,c,Converted from PRECSNOLAND (kg/m2/s) to mm/hr',
                output_file
            ], check=True, capture_output=True)
        except (FileNotFoundError, subprocess.CalledProcessError):
            pass  # ncatted not available, skip metadata
        
        # Clean up temp files
        for f in [subset_file, calc_file, final_file]:
            if Path(f).exists():
                Path(f).unlink()
        
        return True
        
    except subprocess.CalledProcessError as e:
        print(f"CDO error: {e.stderr.decode() if e.stderr else str(e)}")
        return False
    except Exception as e:
        print(f"Error: {str(e)}")
        return False


def process_single_day_cdo(date, bbox, cdo_bbox, collection_id, output_dir, temp_dir, auth):
    """
    Complete workflow for processing a single day with CDO.
    """
    if isinstance(date, str):
        date = pd.to_datetime(date)
    
    date_str = date.strftime('%Y-%m-%d')
    output_file = output_dir / f"merra2_soil_precip_us_{date.strftime('%Y%m%d')}.nc"
    
    # Check if already processed
    if output_file.exists():
        return {'success': True, 'message': 'Already exists', 'skipped': True}
    
    try:
        # Download file
        input_file = download_merra2_file(date, collection_id, auth)
        
        if input_file is None:
            return {'success': False, 'message': f'No data found for {date_str}'}
        
        # Process with CDO
        success = process_soil_precip_cdo(input_file, output_file, cdo_bbox)
        
        # Clean up downloaded file
        if Path(input_file).exists():
            Path(input_file).unlink()
        
        if success:
            return {'success': True, 'message': f'Processed {date_str}'}
        else:
            return {'success': False, 'message': f'CDO processing failed for {date_str}'}
            
    except Exception as e:
        return {'success': False, 'message': f'Error: {str(e)}'}


print("Functions defined successfully")

Functions defined successfully


## 6. Process Single Day (Test)

In [ ]:
# # Test with a single day
# test_date = "2023-06-01"

# print(f"Testing with {test_date}...\n")

# result = process_single_day_cdo(
#     date=test_date,
#     bbox=bbox,
#     cdo_bbox=cdo_bbox,
#     collection_id=collection_id,
#     output_dir=output_dir,
#     temp_dir=temp_dir,
#     auth=auth
# )

# print(f"Result: {result['message']}")

# if result['success'] and not result.get('skipped', False):
#     # Verify output file
#     import xarray as xr
#     test_file = output_dir / f"merra2_soil_precip_us_{pd.to_datetime(test_date).strftime('%Y%m%d')}.nc"
#     ds = xr.open_dataset(test_file)
#     print(f"\nOutput file info:")
#     print(f"  Variables: {list(ds.data_vars)}")
#     print(f"  Dimensions: {dict(ds.dims)}")
#     print(f"  File size: {test_file.stat().st_size / (1024**2):.2f} MB")
    
#     # Show variable details
#     for var in ds.data_vars:
#         print(f"\n  {var}:")
#         print(f"    Units: {ds[var].attrs.get('units', 'N/A')}")
#         print(f"    Long name: {ds[var].attrs.get('long_name', 'N/A')}")
#         print(f"    Data type: {ds[var].dtype}")
#         print(f"    Shape: {ds[var].shape}")
        
#         # Compute min/max safely
#         try:
#             var_min = float(ds[var].min().values)
#             var_max = float(ds[var].max().values)
#             var_mean = float(ds[var].mean().values)
#             print(f"    Range: {var_min:.4f} to {var_max:.4f}")
#             print(f"    Mean: {var_mean:.4f}")
#         except (ValueError, TypeError) as e:
#             print(f"    Range: Could not compute (error: {e})")
    
#     ds.close()

Testing with 2023-06-01...



QUEUEING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

PROCESSING TASKS | :   0%|          | 0/1 [00:00<?, ?it/s]

COLLECTING RESULTS | :   0%|          | 0/1 [00:00<?, ?it/s]

Result: Processed 2023-06-01

Output file info:
  Variables: ['GWETROOT', 'quantization_info', 'GWETTOP', 'PRECIP_SNOW', 'PRECIP_TOTAL', 'PRMC', 'RZMC', 'SFMC']
  Dimensions: {'time': 24, 'lat': 51, 'lon': 95}
  File size: 1.34 MB

  GWETROOT:
    Units: 1
    Long name: root_zone_soil_wetness
    Data type: float32
    Shape: (24, 51, 95)
    Range: 0.1972 to 0.9155
    Mean: 0.5183

  quantization_info:
    Units: N/A
    Long name: N/A
    Data type: |S1
    Shape: ()
    Range: Could not compute (error: could not convert string to float: b'')

  GWETTOP:
    Units: 1
    Long name: surface_soil_wetness
    Data type: float32
    Shape: (24, 51, 95)
    Range: 0.0845 to 1.0000
    Mean: 0.6532

  PRECIP_SNOW:
    Units: mm/hr
    Long name: Snowfall over land
    Data type: float32
    Shape: (24, 51, 95)
    Range: 0.0000 to 0.7872
    Mean: 0.0002

  PRECIP_TOTAL:
    Units: mm/hr
    Long name: Total precipitation over land
    Data type: float32
    Shape: (24, 51, 95)
    Range

/var/folders/jq/m05tkv2d1llbn0w9wc6cltyr0000gn/T/ipykernel_39419/1341717462.py:25: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  print(f"  Dimensions: {dict(ds.dims)}")


## 7. Process All Days

In [ ]:
# Generate date range
dates = pd.date_range(start_date, end_date, freq='D').tolist()

print(f"Total dates to process: {len(dates):,}\n")

# Check existing files
existing = sum(1 for d in dates if check_file_exists(d, output_dir))
print(f"Already processed: {existing:,}")
print(f"Remaining: {len(dates) - existing:,}\n")

# Process all days
results = {'success': 0, 'failed': 0, 'skipped': 0}

for date in tqdm(dates, desc="Processing MERRA-2 Soil & Precip"):
    result = process_single_day_cdo(
        date=date,
        bbox=bbox,
        cdo_bbox=cdo_bbox,
        collection_id=collection_id,
        output_dir=output_dir,
        temp_dir=temp_dir,
        auth=auth
    )
    
    if result['success']:
        if result.get('skipped', False):
            results['skipped'] += 1
        else:
            results['success'] += 1
            if results['success'] % 100 == 0:
                print(f"Processed {results['success']} files...")
    else:
        results['failed'] += 1
        print(f"✗ {date.strftime('%Y-%m-%d')}: {result['message']}")

print("\n" + "="*60)
print("PROCESSING COMPLETE")
print("="*60)
print(f"Successfully processed: {results['success']:,}")
print(f"Skipped (existing): {results['skipped']:,}")
print(f"Failed: {results['failed']:,}")

## 8. Process Date Range (Optional)

In [ ]:
# Process a specific month or year
test_start = "2023-06-01"
test_end = "2023-06-30"

test_dates = pd.date_range(test_start, test_end, freq='D').tolist()
print(f"Processing {len(test_dates)} days from {test_start} to {test_end}\n")

for date in tqdm(test_dates, desc="Processing test range"):
    result = process_single_day_cdo(
        date=date,
        bbox=bbox,
        cdo_bbox=cdo_bbox,
        collection_id=collection_id,
        output_dir=output_dir,
        temp_dir=temp_dir,
        auth=auth
    )
    
    status = "✓" if result['success'] else "✗"
    print(f"{status} {date.strftime('%Y-%m-%d')}: {result['message']}")

## 9. Clean Up Temp Directory

In [ ]:
# Clean up any remaining temp files
import shutil

temp_files = list(temp_dir.glob("*"))
if len(temp_files) > 0:
    print(f"Cleaning up {len(temp_files)} temp files...")
    for f in temp_files:
        if f.is_file():
            f.unlink()
    print("✓ Temp directory cleaned")
else:
    print("✓ Temp directory already clean")

## Variable Reference

### Soil Moisture Variables (from M2T1NXLND):

| Variable | Description | Units | Depth | Range |
|----------|-------------|-------|-------|-------|
| **SFMC** | Surface soil moisture content | kg/m² | 0-5 cm | 0-50 |
| **RZMC** | Root zone soil moisture content | kg/m² | 0-100 cm | 0-1000 |
| **PRMC** | Profile soil moisture content | kg/m² | Full column | 0-2000 |
| **GWETROOT** | Root zone soil wetness | fraction | 0-100 cm | 0-1 |
| **GWETTOP** | Surface soil wetness | fraction | 0-5 cm | 0-1 |

### Soil Temperature Layers:

| Variable | Description | Original Units | Output Units | Depth |
|----------|-------------|----------------|--------------|-------|
| **TSOIL1** | Soil temperature layer 1 | K | °C | ~0-10 cm |
| **TSOIL2** | Soil temperature layer 2 | K | °C | ~10-40 cm |
| **TSOIL3** | Soil temperature layer 3 | K | °C | ~40-100 cm |
| **TSOIL4** | Soil temperature layer 4 | K | °C | ~100-200 cm |

### Precipitation Variables:

| Variable | Description | Original Units | Output Units |
|----------|-------------|----------------|-------------|
| **PRECIP** | Bias-corrected precipitation | kg/m²/s | mm/hr |

### Conversion Notes:

**Soil Temperature:**
- Formula: `TSOIL (°C) = TSOIL (K) - 273.15`

**Precipitation:**
- 1 kg/m²/s = 1 mm/s (water density = 1000 kg/m³)
- mm/hr = mm/s × 3600
- Formula: `PRECIP (mm/hr) = PRECTOTCORR (kg/m²/s) × 3600`

**Soil Moisture Interpretation:**

*Wetness Fractions (GWET variables):*
- 0.0 = Completely dry (wilting point)
- 0.5 = 50% saturated
- 1.0 = Fully saturated (field capacity)

*Moisture Content (MC variables in kg/m²):*
- Represents total water mass per unit area in each layer
- Higher values = more water stored in soil
- Varies by soil type, depth, and texture

### Layer Depths:

MERRA-2 uses 4 soil layers with approximate depths:
1. **Layer 1** (~0-10 cm): Surface/topsoil
2. **Layer 2** (~10-40 cm): Upper root zone
3. **Layer 3** (~40-100 cm): Deep root zone
4. **Layer 4** (~100-200 cm): Subsoil/deep profile

### Data Characteristics:
- **Temporal Resolution**: Hourly (time-averaged)
- **Spatial Resolution**: 0.625° longitude × 0.5° latitude (~50-70 km)
- **Time Coverage**: 1980-present (with ~2-3 month latency)
- **Collection**: M2T1NXLND v5.12.4